In [2]:
import os
import requests
from dotenv import load_dotenv
import sqlite3
import pandas as pd
import time

In [5]:
load_dotenv()

API_KEY = os.getenv('WEBACY_API_KEY') # Fetching the Etherscan API key from environment variables

if not API_KEY: 
    raise ValueError('Please set the WEBACY_API_KEY environment variable.')

In [ ]:
address = "0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48" # ERC 20 USDC address

url = f"https://api.webacy.com/addresses/{address}"

headers = {
    "accept": "application/json",
    "x-api-key": API_KEY
}

response = requests.get(url, headers=headers)

try:
    data = response.json()
except ValueError:
    data = response.text

data


In [12]:
print(type(data))
print(data)


<class 'dict'>
{'count': 6, 'medium': 0, 'high': 0, 'overallRisk': 10.488088481701514, 'issues': [{'score': 0.21999999999999997, 'tags': [{'name': 'Verified Listing', 'description': 'This asset represents an established project with proven authenticity or reputation, and has been verified as authentic. This does not necessarily mean the asset is free of risks or issues.', 'type': 'tokenRisk', 'severity': 0, 'key': 'verified_listing'}, {'name': 'Has Been Sniped', 'description': 'Token shows signs of sniper activity, where automated tools or bots were used to purchase significant amounts of supply within seconds/minutes of minting. This often indicates coordinated buying activity and potential for future price manipulation.', 'type': 'tokenRisk', 'severity': 0, 'key': 'has_been_sniped'}, {'name': 'Proxy', 'description': 'Proxy contracts enable upgradability, meaning the logic or implementation contract that a proxy points to can be changed. The primary risk lies with the owner, who holds

In [3]:
df = pd.json_normalize(
    data,
    record_path=['issues', 'tags'],
    meta=['overallRisk', 'isContract', ['issues', 'score']]
)

NameError: name 'data' is not defined

In [14]:
df = pd.json_normalize(data)
df.head()


,count,medium,high,overallRisk,issues,isContract,details.fund_flows.risk.ofac,details.fund_flows.risk.hacker,details.fund_flows.risk.mixers,details.fund_flows.risk.drainer,...,details.marketData.holderAnalysis.top_10_holders_analysis.topHolders,details.marketData.holderAnalysis.top_10_holders_analysis.totalSupply,details.marketData.holderAnalysis.top_10_holders_analysis.percentageHeldByTop5,details.marketData.holderAnalysis.top_10_holders_analysis.percentageHeldByTop10,details.marketData.holderAnalysis.top_10_holders_analysis.percentageHeldByTop20,details.marketData.holderAnalysis.dev_launched_tokens_in_24_hours,details.access_control.interfaceType,details.access_control.activeRoleHolders,details.buy_sell_taxes.has_buy_tax,details.buy_sell_taxes.has_sell_tax
0,6,0,0,10.488088,"[{'score': 0.21999999999999997, 'tags': [{'nam...",True,False,False,False,False,...,"[{'amount': '2382347666.5019202232', 'percenta...",0.04953476450227574,500,1000,2000,0,Ownable,"[{'name': 'OWNER_ROLE', 'role': 'owner', 'acco...",False,False


In [15]:
df_issues = pd.json_normalize(data, record_path=['issues'])
df_issues.head()


,score,tags,riskScore,categories.contract_issues.key,categories.contract_issues.name,categories.contract_issues.gradedDescription.high,categories.contract_issues.gradedDescription.medium,categories.contract_issues.gradedDescription.low,categories.contract_issues.tags.is_proxy,categories.governance_issues.key,categories.governance_issues.name,categories.governance_issues.description,categories.governance_issues.tags.access_control
0,0.22,"[{'name': 'Verified Listing', 'description': '...",Low Risk,contract_issues,Contract Exploits,"Our detectors have found, with high certainty,...",Our detectors have found some logic in this sm...,Our detectors have found issues with this smar...,True,governance_issues,Contract Governance,Suspect logic in the smart contract for this a...,True


In [20]:
import requests
import json
import time
from datetime import datetime
from typing import Dict, List, Optional, Union
import pandas as pd
import logging

class BlockchainAnomalyTracker:
    """
    Blockchain anomaly tracker using only fields available in Webacy API response
    """
    
    def __init__(self, api_key: str):
        self.api_key = API_KEY
        self.base_url = "https://api.webacy.com"
        self.headers = {
            "accept": "application/json",
            "x-api-key": self.api_key
        }
        
        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger(__name__)
    
    def fetch_address_data(self, address: str) -> Optional[Dict]:
        """
        Fetch data for any address using Webacy API
        """
        url = f"{self.base_url}/addresses/{address}"
        
        try:
            response = requests.get(url, headers=self.headers, timeout=30)
            response.raise_for_status()
            
            data = response.json()
            self.logger.info(f"Successfully fetched data for {address}")
            return data
            
        except requests.exceptions.RequestException as e:
            self.logger.error(f"Error fetching data for {address}: {e}")
            return None
        except json.JSONDecodeError as e:
            self.logger.error(f"JSON decode error for {address}: {e}")
            return None
    
    def analyze_data(self, address: str) -> Dict:
        """
        Analyze address data using only available response fields
        """
        data = self.fetch_address_data(address)
        if not data:
            return {"error": "Failed to fetch data", "address": address}
        
        analysis = {
            'address': address,
            'analysis_timestamp': datetime.now().isoformat(),
            'isContract': data.get('isContract', False),
            'overall_risk': data.get('overallRisk', 0),
            'issue_count': data.get('count', 0),
            'medium_count': data.get('medium', 0),
            'high_count': data.get('high', 0)
        }
        
        # Extract issues data
        issues_list = data.get('issues', [])
        if issues_list:
            issue_data = issues_list[0]  # First issue group
            analysis['risk_score'] = issue_data.get('score', 0)
            analysis['risk_level'] = issue_data.get('riskScore', 'Unknown')
            
            # Extract tags
            tags = issue_data.get('tags', [])
            analysis['risk_tags'] = []
            for tag in tags:
                analysis['risk_tags'].append({
                    'name': tag.get('name', ''),
                    'severity': tag.get('severity', 0),
                    'key': tag.get('key', '')
                })
            
            # Extract categories
            categories = issue_data.get('categories', {})
            analysis['categories'] = {}
            for cat_key, cat_data in categories.items():
                analysis['categories'][cat_key] = {
                    'name': cat_data.get('name', ''),
                    'key': cat_data.get('key', '')
                }
        
        # Extract token information if available
        details = data.get('details', {})
        if 'token_info' in details:
            token_info = details['token_info']
            analysis['token_info'] = {
                'name': token_info.get('name', ''),
                'symbol': token_info.get('symbol', ''),
                'decimals': token_info.get('decimals', 0),
                'description': token_info.get('description', '')
            }
        
        # Extract address info
        if 'address_info' in details:
            addr_info = details['address_info']
            analysis['address_info'] = {
                'balance': addr_info.get('balance', 0),
                'transaction_count': addr_info.get('transaction_count', 0),
                'time_1st_tx': addr_info.get('time_1st_tx', ''),
                'ofac_sanctioned': addr_info.get('ofac_sanctioned', False),
                'dprk': addr_info.get('dprk', False),
                'has_no_balance': addr_info.get('has_no_balance', True),
                'has_no_transactions': addr_info.get('has_no_transactions', False)
            }
        
        # Extract fund flow risks
        if 'fund_flows' in details:
            fund_flows = details['fund_flows']
            analysis['fund_flows'] = {
                'label': fund_flows.get('label', ''),
                'risk_flags': fund_flows.get('risk', {})
            }
        
        # Extract market data if available
        if 'marketData' in details:
            market_data = details['marketData']
            analysis['market_data'] = {
                'current_price': market_data.get('current_price', 0),
                'market_cap': market_data.get('market_cap', 0),
                'total_supply': market_data.get('total_supply', 0),
                'circulating_supply': market_data.get('circulating_supply', 0),
                'price_change_24h': market_data.get('price_change_24h', 0),
                'price_change_percentage_24h': market_data.get('price_change_percentage_24h', 0)
            }
            
            # Ownership distribution
            ownership = market_data.get('ownershipDistribution', {})
            if ownership:
                analysis['ownership'] = {
                    'totalSupply': ownership.get('totalSupply', 0),
                    'percentageHeldByTop10': ownership.get('percentageHeldByTop10', 0),
                    'top_holders_count': len(ownership.get('topHolders', []))
                }
            
            # Holder analysis
            holder_analysis = market_data.get('holderAnalysis', {})
            if holder_analysis:
                sniper_analysis = holder_analysis.get('sniper_analysis', {})
                analysis['sniper_data'] = {
                    'sniper_count': sniper_analysis.get('sniper_count', 0),
                    'sniper_total_percentage': sniper_analysis.get('sniper_total_percentage', 0),
                    'sniper_confidence_score': sniper_analysis.get('sniper_confidence_score', 0)
                }
                
                analysis['holder_stats'] = {
                    'total_holders_count': holder_analysis.get('total_holders_count', 0),
                    'token_mint_time': holder_analysis.get('token_mint_time', ''),
                    'minter': holder_analysis.get('minter', '')
                }
        
        return analysis
    
    def monitor_addresses(self, addresses: List[str], interval_minutes: int = 60) -> None:
        """
        Monitor multiple addresses continuously
        """
        self.logger.info(f"Starting monitoring of {len(addresses)} addresses")
        
        while True:
            for address in addresses:
                try:
                    analysis = self.analyze_data(address)
                    
                    # Simple alerting based on available data
                    risk_score = analysis.get('overall_risk', 0)
                    if risk_score > 50:
                        self.logger.warning(f"HIGH RISK: {address} - Risk Score: {risk_score}")
                    
                    # Check for sanctions
                    addr_info = analysis.get('address_info', {})
                    if addr_info.get('ofac_sanctioned', False):
                        self.logger.critical(f"SANCTIONED ADDRESS: {address}")
                    
                    time.sleep(2)
                    
                except Exception as e:
                    self.logger.error(f"Error monitoring {address}: {e}")
            
            time.sleep(interval_minutes * 60)
    
    def generate_report(self, addresses: List[str]) -> pd.DataFrame:
        """
        Generate report for multiple addresses using available data fields
        """
        results = []
        
        for address in addresses:
            try:
                analysis = self.analyze_data(address)
                
                row = {
                    'Address': address,
                    'IsContract': analysis.get('isContract', False),
                    'OverallRisk': analysis.get('overall_risk', 0),
                    'IssueCount': analysis.get('issue_count', 0),
                    'HighCount': analysis.get('high_count', 0),
                    'MediumCount': analysis.get('medium_count', 0),
                    'RiskLevel': analysis.get('risk_level', 'Unknown')
                }
                
                # Add token info if available
                token_info = analysis.get('token_info', {})
                if token_info:
                    row['TokenName'] = token_info.get('name', '')
                    row['TokenSymbol'] = token_info.get('symbol', '')
                
                # Add address info
                addr_info = analysis.get('address_info', {})
                if addr_info:
                    row['OFAC_Sanctioned'] = addr_info.get('ofac_sanctioned', False)
                    row['TransactionCount'] = addr_info.get('transaction_count', 0)
                    row['HasBalance'] = not addr_info.get('has_no_balance', True)
                
                # Add ownership data
                ownership = analysis.get('ownership', {})
                if ownership:
                    row['Top10Percentage'] = ownership.get('percentageHeldByTop10', 0)
                
                # Add sniper data
                sniper_data = analysis.get('sniper_data', {})
                if sniper_data:
                    row['SniperCount'] = sniper_data.get('sniper_count', 0)
                    row['SniperPercentage'] = sniper_data.get('sniper_total_percentage', 0)
                
                results.append(row)
                
            except Exception as e:
                self.logger.error(f"Error analyzing {address}: {e}")
                results.append({
                    'Address': address,
                    'Error': str(e)
                })
        
        return pd.DataFrame(results)
    
    def get_real_time_data(self, address: str) -> Dict:
        """
        Get real-time data for a single address
        """
        return self.analyze_data(address)

# Usage example
def main():
    API_KEY = "your_webacy_api_key_here"
    tracker = BlockchainAnomalyTracker(API_KEY)
    
    # Single address real-time analysis
    address = "0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48"
    result = tracker.get_real_time_data(address)
    
    print("Real-time Analysis:")
    print(f"Address: {result.get('address')}")
    print(f"Overall Risk: {result.get('overall_risk')}")
    print(f"Risk Level: {result.get('risk_level')}")
    print(f"Issue Count: {result.get('issue_count')}")
    
    # Batch analysis
    test_addresses = [
        "0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48",
        "0xdAC17F958D2ee523a2206206994597C13D831ec7"
    ]
    
    report = tracker.generate_report(test_addresses)
    print("\nBatch Report:")
    print(report.to_string(index=False))

if __name__ == "__main__":
    main()

INFO:__main__:Successfully fetched data for 0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48


Real-time Analysis:
Address: 0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48
Overall Risk: 10.488088481701514
Risk Level: Low Risk
Issue Count: 6


INFO:__main__:Successfully fetched data for 0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48
ERROR:__main__:Error fetching data for 0xdAC17F958D2ee523a2206206994597C13D831ec7: 504 Server Error: Gateway Timeout for url: https://api.webacy.com/addresses/0xdAC17F958D2ee523a2206206994597C13D831ec7



Batch Report:
                                   Address  IsContract  OverallRisk  IssueCount  HighCount  MediumCount RiskLevel TokenName TokenSymbol OFAC_Sanctioned  TransactionCount HasBalance  Top10Percentage  SniperCount  SniperPercentage
0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48        True    10.488088           6          0            0  Low Risk      USDC        USDC           False             101.0      False            39.37         17.0         35.020428
0xdAC17F958D2ee523a2206206994597C13D831ec7       False     0.000000           0          0            0   Unknown       NaN         NaN             NaN               NaN        NaN              NaN          NaN               NaN


In [21]:
import requests
import time
import json
import os
from datetime import datetime, timedelta
from typing import Dict, List, Optional
import logging
from dotenv import load_dotenv

class DetailedContractAnalyzer:
    def __init__(self, api_key: str):
        """
        Initialize the detailed contract analyzer
        
        Args:
            api_key: Webacy API key
        """
        self.api_key = api_key
        self.base_url = "https://api.webacy.com/contracts"
        
        # Setup logging
        logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
        self.logger = logging.getLogger(__name__)

    def fetch_contract_data(self, contract_address: str) -> Optional[Dict]:
        """Fetch contract data from Webacy API"""
        url = f"{self.base_url}/{contract_address}"
        
        headers = {
            "accept": "application/json",
            "x-api-key": self.api_key
        }
        
        try:
            response = requests.get(url, headers=headers)
            
            try:
                data = response.json()
            except ValueError:
                data = response.text
                self.logger.error(f"Invalid JSON response for {contract_address}: {data}")
                return None
                
            if response.status_code != 200:
                self.logger.error(f"API error {response.status_code} for {contract_address}: {data}")
                return None
                
            return data
            
        except requests.exceptions.RequestException as e:
            self.logger.error(f"Error fetching data for {contract_address}: {e}")
            return None

    def analyze_risk_assessment(self, data: Dict) -> Dict:
        """Extract and analyze risk assessment data"""
        risk_analysis = {
            'overall_risk_score': data.get('riskScore', 'Unknown'),
            'analysis_status': data.get('analysis_status', 'Unknown'),
            'analysis_type': data.get('analysis_type', 'Unknown'),
            'score': data.get('score', 0),
            'tags': data.get('tags', []),
            'categories': data.get('categories', {}),
        }
        
        # Source code analysis details
        source_analysis = data.get('source_code_analysis', {})
        if source_analysis:
            analysis_details = source_analysis.get('analysis', {})
            risk_analysis.update({
                'contract_address': analysis_details.get('contract_address', 'N/A'),
                'blockchain': analysis_details.get('chain', 'Unknown'),
                'analysis_date': analysis_details.get('analysis_date', 'Unknown'),
                'analysis_status_detailed': analysis_details.get('status', 'Unknown'),
                'findings': analysis_details.get('findings', []),
                'reference_urls': analysis_details.get('urls', [])
            })
        
        return risk_analysis

    def analyze_market_data(self, data: Dict) -> Dict:
        """Extract and analyze comprehensive market data"""
        metadata = data.get('metadata', {})
        market_data = metadata.get('market_data', {})
        
        if not market_data:
            return {'error': 'No market data available'}
        
        # Price analysis
        current_price = market_data.get('current_price', 0)
        price_change_24h = market_data.get('price_change_24h', 0)
        price_change_pct_24h = market_data.get('price_change_percentage_24h', 0)
        
        # Market cap analysis
        market_cap = market_data.get('market_cap', 0)
        market_cap_rank = market_data.get('market_cap_rank', 'N/A')
        market_cap_change_24h = market_data.get('market_cap_change_24h', 0)
        market_cap_change_pct_24h = market_data.get('market_cap_change_percentage_24h', 0)
        
        # Supply analysis
        circulating_supply = market_data.get('circulating_supply', 0)
        total_supply = market_data.get('total_supply', 0)
        max_supply = market_data.get('max_supply', 'Unlimited')
        
        # Historical performance
        ath = market_data.get('ath', 0)
        ath_date = market_data.get('ath_date', 'Unknown')
        ath_change_pct = market_data.get('ath_change_percentage', 0)
        atl = market_data.get('atl', 0)
        atl_date = market_data.get('atl_date', 'Unknown')
        atl_change_pct = market_data.get('atl_change_percentage', 0)
        
        # Volume and liquidity
        total_volume = market_data.get('total_volume', 0)
        fdv = market_data.get('fully_diluted_valuation', 0)
        
        return {
            'token_info': {
                'name': market_data.get('name', 'Unknown'),
                'symbol': market_data.get('symbol', 'Unknown').upper(),
                'image': market_data.get('image', 'N/A'),
                'last_updated': market_data.get('last_updated', 'Unknown')
            },
            'current_metrics': {
                'price': f"${current_price:,.2f}",
                'price_change_24h': f"${price_change_24h:,.2f}",
                'price_change_percentage_24h': f"{price_change_pct_24h:.2f}%",
                'market_cap': f"${market_cap:,.0f}",
                'market_cap_rank': market_cap_rank,
                'volume_24h': f"${total_volume:,.0f}"
            },
            'supply_metrics': {
                'circulating_supply': f"{circulating_supply:,.2f}",
                'total_supply': f"{total_supply:,.2f}",
                'max_supply': max_supply if max_supply else "Unlimited",
                'fully_diluted_valuation': f"${fdv:,.0f}"
            },
            'historical_performance': {
                'all_time_high': f"${ath:,.2f}",
                'ath_date': ath_date,
                'ath_change_percentage': f"{ath_change_pct:.2f}%",
                'all_time_low': f"${atl:.2f}",
                'atl_date': atl_date,
                'atl_change_percentage': f"{atl_change_pct:.2f}%"
            },
            'market_changes_24h': {
                'market_cap_change': f"${market_cap_change_24h:,.0f}",
                'market_cap_change_percentage': f"{market_cap_change_pct_24h:.2f}%"
            }
        }

    def analyze_price_trends(self, data: Dict) -> Dict:
        """Analyze price trends from sparkline data"""
        metadata = data.get('metadata', {})
        market_data = metadata.get('market_data', {})
        sparkline = market_data.get('sparkline_in_7d', {})
        prices = sparkline.get('price', [])
        
        if not prices:
            return {'error': 'No price trend data available'}
        
        # Calculate trend metrics
        first_price = prices[0]
        last_price = prices[-1]
        max_price = max(prices)
        min_price = min(prices)
        avg_price = sum(prices) / len(prices)
        
        # Calculate volatility (standard deviation)
        variance = sum((p - avg_price) ** 2 for p in prices) / len(prices)
        volatility = variance ** 0.5
        volatility_pct = (volatility / avg_price) * 100
        
        # Trend direction
        week_change = ((last_price - first_price) / first_price) * 100
        trend_direction = "Bullish" if week_change > 0 else "Bearish" if week_change < 0 else "Sideways"
        
        return {
            '7_day_summary': {
                'starting_price': f"${first_price:.2f}",
                'ending_price': f"${last_price:.2f}",
                'week_change': f"{week_change:.2f}%",
                'trend_direction': trend_direction
            },
            'price_extremes': {
                'highest_price_7d': f"${max_price:.2f}",
                'lowest_price_7d': f"${min_price:.2f}",
                'price_range': f"${max_price - min_price:.2f}",
                'range_percentage': f"{((max_price - min_price) / min_price) * 100:.2f}%"
            },
            'volatility_analysis': {
                'average_price_7d': f"${avg_price:.2f}",
                'volatility': f"${volatility:.2f}",
                'volatility_percentage': f"{volatility_pct:.2f}%",
                'volatility_rating': self.get_volatility_rating(volatility_pct)
            },
            'data_points': len(prices)
        }

    def get_volatility_rating(self, volatility_pct: float) -> str:
        """Rate volatility based on percentage"""
        if volatility_pct < 2:
            return "Very Low"
        elif volatility_pct < 5:
            return "Low"
        elif volatility_pct < 10:
            return "Moderate"
        elif volatility_pct < 20:
            return "High"
        else:
            return "Very High"

    def analyze_deployer_info(self, data: Dict) -> Dict:
        """Analyze deployer information"""
        deployer = data.get('deployer', {})
        deployed_contracts = deployer.get('deployed_contracts', [])
        
        return {
            'total_deployed_contracts': len(deployed_contracts),
            'deployed_contracts': deployed_contracts[:10],  # Show first 10
            'has_multiple_deployments': len(deployed_contracts) > 1
        }

    def generate_comprehensive_report(self, contract_address: str) -> Dict:
        """Generate a comprehensive contract analysis report"""
        self.logger.info(f"Generating comprehensive report for {contract_address}")
        
        data = self.fetch_contract_data(contract_address)
        if not data:
            return {'error': 'Failed to fetch contract data'}
        
        report = {
            'contract_address': contract_address,
            'analysis_timestamp': datetime.now().isoformat(),
            'risk_assessment': self.analyze_risk_assessment(data),
            'market_analysis': self.analyze_market_data(data),
            'price_trends': self.analyze_price_trends(data),
            'deployer_analysis': self.analyze_deployer_info(data),
            'raw_data_summary': {
                'total_fields': len(data),
                'has_market_data': 'metadata' in data and 'market_data' in data.get('metadata', {}),
                'has_risk_data': 'riskScore' in data,
                'has_analysis_data': 'source_code_analysis' in data,
                'data_completeness': self.calculate_data_completeness(data)
            }
        }
        
        return report

    def calculate_data_completeness(self, data: Dict) -> str:
        """Calculate how complete the data response is"""
        expected_fields = [
            'riskScore', 'metadata', 'source_code_analysis', 
            'deployer', 'analysis_status', 'score'
        ]
        
        present_fields = sum(1 for field in expected_fields if field in data)
        completeness_pct = (present_fields / len(expected_fields)) * 100
        
        if completeness_pct >= 90:
            return f"Excellent ({completeness_pct:.0f}%)"
        elif completeness_pct >= 70:
            return f"Good ({completeness_pct:.0f}%)"
        elif completeness_pct >= 50:
            return f"Fair ({completeness_pct:.0f}%)"
        else:
            return f"Poor ({completeness_pct:.0f}%)"

    def print_formatted_report(self, report: Dict):
        """Print a beautifully formatted report"""
        if 'error' in report:
            print(f"❌ Error: {report['error']}")
            return
        
        print("=" * 80)
        print(f"🔍 COMPREHENSIVE CONTRACT ANALYSIS REPORT")
        print("=" * 80)
        print(f"📋 Contract: {report['contract_address']}")
        print(f"🕒 Analysis Time: {report['analysis_timestamp']}")
        print()
        
        # Risk Assessment
        risk = report['risk_assessment']
        print("🛡️  RISK ASSESSMENT")
        print("-" * 40)
        print(f"Overall Risk Score: {risk['overall_risk_score']}")
        print(f"Security Score: {risk['score']}")
        print(f"Analysis Status: {risk['analysis_status_detailed']}")
        print(f"Blockchain: {risk['blockchain']}")
        if risk['tags']:
            print(f"Tags: {', '.join(risk['tags'])}")
        print()
        
        # Market Analysis
        market = report['market_analysis']
        if 'error' not in market:
            token_info = market['token_info']
            current = market['current_metrics']
            supply = market['supply_metrics']
            historical = market['historical_performance']
            
            print("💰 MARKET ANALYSIS")
            print("-" * 40)
            print(f"Token: {token_info['name']} ({token_info['symbol']})")
            print(f"Current Price: {current['price']}")
            print(f"24h Change: {current['price_change_24h']} ({current['price_change_percentage_24h']})")
            print(f"Market Cap: {current['market_cap']} (Rank #{current['market_cap_rank']})")
            print(f"24h Volume: {current['volume_24h']}")
            print()
            
            print("📊 SUPPLY METRICS")
            print("-" * 40)
            print(f"Circulating Supply: {supply['circulating_supply']}")
            print(f"Total Supply: {supply['total_supply']}")
            print(f"Max Supply: {supply['max_supply']}")
            print(f"Fully Diluted Valuation: {supply['fully_diluted_valuation']}")
            print()
            
            print("📈 HISTORICAL PERFORMANCE")
            print("-" * 40)
            print(f"All-Time High: {historical['all_time_high']} ({historical['ath_change_percentage']} from ATH)")
            print(f"All-Time Low: {historical['all_time_low']} ({historical['atl_change_percentage']} from ATL)")
            print()
        
        # Price Trends
        trends = report['price_trends']
        if 'error' not in trends:
            print("📉 7-DAY PRICE TRENDS")
            print("-" * 40)
            summary = trends['7_day_summary']
            extremes = trends['price_extremes']
            volatility = trends['volatility_analysis']
            
            print(f"Week Performance: {summary['week_change']} ({summary['trend_direction']})")
            print(f"Price Range: {extremes['lowest_price_7d']} - {extremes['highest_price_7d']}")
            print(f"Volatility: {volatility['volatility_percentage']} ({volatility['volatility_rating']})")
            print()
        
        # Deployer Analysis
        deployer = report['deployer_analysis']
        print("👤 DEPLOYER ANALYSIS")
        print("-" * 40)
        print(f"Total Deployed Contracts: {deployer['total_deployed_contracts']}")
        print(f"Multiple Deployments: {'Yes' if deployer['has_multiple_deployments'] else 'No'}")
        print()
        
        # Data Quality
        summary = report['raw_data_summary']
        print("📋 DATA COMPLETENESS")
        print("-" * 40)
        print(f"Data Quality: {summary['data_completeness']}")
        print(f"Market Data Available: {'✅' if summary['has_market_data'] else '❌'}")
        print(f"Risk Data Available: {'✅' if summary['has_risk_data'] else '❌'}")
        print(f"Analysis Data Available: {'✅' if summary['has_analysis_data'] else '❌'}")
        print("=" * 80)

# Usage Example
if __name__ == "__main__":
    # Load environment variables
    load_dotenv()
    
    # Fetch API key from environment variables
    API_KEY = os.getenv('WEBACY_API_KEY')
    
    if not API_KEY: 
        raise ValueError('Please set the WEBACY_API_KEY environment variable.')
    
    # Initialize the analyzer
    analyzer = DetailedContractAnalyzer(API_KEY)
    
    # Analyze the WETH contract
    contractAddress = "0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2"  # WETH
    
    # Generate comprehensive report
    report = analyzer.generate_comprehensive_report(contractAddress)
    
    # Print formatted report
    analyzer.print_formatted_report(report)
    
    # Also save raw JSON for detailed analysis
    with open(f'contract_analysis_{contractAddress}.json', 'w') as f:
        json.dump(report, f, indent=2)
    
    print(f"\n💾 Detailed JSON report saved to: contract_analysis_{contractAddress}.json")

INFO:__main__:Generating comprehensive report for 0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2


🔍 COMPREHENSIVE CONTRACT ANALYSIS REPORT
📋 Contract: 0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
🕒 Analysis Time: 2025-09-17T22:41:51.283962

🛡️  RISK ASSESSMENT
----------------------------------------
Overall Risk Score: Low Risk
Security Score: 0
Analysis Status: pending
Blockchain: eth

💰 MARKET ANALYSIS
----------------------------------------
Token: Unknown (UNKNOWN)
Current Price: $4,523.78
24h Change: $30.80 (0.69%)
Market Cap: $11,137,598,313 (Rank #24)
24h Volume: $373,979,900

📊 SUPPLY METRICS
----------------------------------------
Circulating Supply: 2,462,041.06
Total Supply: 2,462,041.06
Max Supply: Unlimited
Fully Diluted Valuation: $11,137,598,313

📈 HISTORICAL PERFORMANCE
----------------------------------------
All-Time High: $4,950.08 (-8.70% from ATH)
All-Time Low: $82.10 (5404.80% from ATL)

📉 7-DAY PRICE TRENDS
----------------------------------------
Week Performance: 3.97% (Bullish)
Price Range: $4319.35 - $4761.46
Volatility: 2.22% (Low)

👤 DEPLOYER ANALYSIS
-

In [7]:
import requests
import time
import re
import os
from typing import Dict, Optional, Union
from datetime import datetime, timedelta
from dotenv import load_dotenv
import json

class UniswapV3RiskTracker:
    """
    Optimized tracker for monitoring threat risk of EOA and contract interactions
    on Uniswap V3 router using Webacy API
    """
    
    def __init__(self, api_key: str, cache_duration_minutes: int = 30):
        """
        Initialize the risk tracker
        
        Args:
            api_key: Webacy API key
            cache_duration_minutes: How long to cache results (default 30 minutes)
        """
        if not api_key or api_key == "your_webacy_api_key_here":
            raise ValueError("Valid API key is required. Please set your actual Webacy API key.")
        
        self.api_key = api_key
        self.cache_duration = timedelta(minutes=cache_duration_minutes)
        self.cache = {}
        self.session = requests.Session()
        self.base_url = "https://api.webacy.com/addresses"
        
        # Set up session headers - using exact format from your working code
        self.session.headers.update({
            "accept": "application/json",
            "x-api-key": self.api_key
        })
        
        # Rate limiting (adjust based on your API plan)
        self.last_request_time = 0
        self.min_request_interval = 0.1  # 100ms between requests
    
    def validate_ethereum_address(self, address: str) -> bool:
        """
        Validate if the provided string is a valid Ethereum address
        
        Args:
            address: The address to validate
            
        Returns:
            bool: True if valid Ethereum address
        """
        if not isinstance(address, str):
            return False
        
        # Remove '0x' prefix if present and check format
        clean_address = address.lower()
        if clean_address.startswith('0x'):
            clean_address = clean_address[2:]
        
        # Check if it's 40 hex characters
        return len(clean_address) == 40 and re.match('^[0-9a-f]+$', clean_address)
    
    def _rate_limit(self):
        """Implement simple rate limiting"""
        current_time = time.time()
        time_since_last = current_time - self.last_request_time
        
        if time_since_last < self.min_request_interval:
            time.sleep(self.min_request_interval - time_since_last)
        
        self.last_request_time = time.time()
    
    def _is_cache_valid(self, address: str) -> bool:
        """Check if cached data is still valid"""
        if address not in self.cache:
            return False
        
        cache_time = self.cache[address]['timestamp']
        return datetime.now() - cache_time < self.cache_duration
    
    def test_api_connection(self) -> Dict:
        """
        Test API connection and authentication
        
        Returns:
            Dict: Connection test results
        """
        test_address = "0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2"  # WETH
        url = f"{self.base_url}/{test_address}"
        
        try:
            # Use the same method as your working code
            headers = {
                "accept": "application/json",
                "x-api-key": self.api_key
            }
            response = requests.get(url, headers=headers, timeout=10)
            
            return {
                "success": response.status_code == 200,
                "status_code": response.status_code,
                "headers_sent": headers,
                "response_preview": response.text[:200] if response.text else "No response body",
                "api_key_preview": self.api_key[:10] + "..." if len(self.api_key) > 10 else "short_key"
            }
        except Exception as e:
            return {
                "success": False,
                "error": str(e),
                "headers_sent": {
                    "accept": "application/json",
                    "x-api-key": self.api_key[:10] + "..."
                }
            }
    
    def get_address_risk_data(self, address: str, force_refresh: bool = False) -> Dict:
        """
        Get comprehensive risk data for any EOA or contract address
        
        Args:
            address: Ethereum address to analyze
            force_refresh: Skip cache and fetch fresh data
            
        Returns:
            Dict: Risk analysis data or error information
        """
        
        # Validate address format
        if not self.validate_ethereum_address(address):
            return {
                "error": "Invalid Ethereum address format",
                "address": address,
                "timestamp": datetime.now().isoformat()
            }
        
        # Normalize address (ensure lowercase with 0x prefix)
        normalized_address = address.lower()
        if not normalized_address.startswith('0x'):
            normalized_address = '0x' + normalized_address
        
        # Check cache first (unless force refresh)
        if not force_refresh and self._is_cache_valid(normalized_address):
            cached_data = self.cache[normalized_address]['data'].copy()
            cached_data['data_source'] = 'cache'
            return cached_data
        
        # Rate limiting
        self._rate_limit()
        
        # Make API request using the exact same method as your working code
        url = f"{self.base_url}/{normalized_address}"
        headers = {
            "accept": "application/json",
            "x-api-key": self.api_key
        }
        
        try:
            response = requests.get(url, headers=headers, timeout=30)
            response.raise_for_status()
            
            # Parse JSON response - same as your working code
            try:
                raw_data = response.json()
            except ValueError:
                raw_data = response.text
                return {
                    "error": "API returned non-JSON response",
                    "address": normalized_address,
                    "timestamp": datetime.now().isoformat(),
                    "raw_response": raw_data[:500]  # First 500 chars
                }
            
            # Process and enrich the data
            processed_data = self._process_risk_data(raw_data, normalized_address)
            
            # Cache the result
            self.cache[normalized_address] = {
                'data': processed_data,
                'timestamp': datetime.now()
            }
            
            processed_data['data_source'] = 'api'
            return processed_data
            
        except requests.exceptions.Timeout:
            return {
                "error": "API request timeout",
                "address": normalized_address,
                "timestamp": datetime.now().isoformat()
            }
        except requests.exceptions.HTTPError as e:
            error_details = ""
            status_code = "Unknown"
            if hasattr(e, 'response') and e.response:
                error_details = e.response.text
                status_code = e.response.status_code
                
                # Special handling for 401 errors
                if e.response.status_code == 401:
                    return {
                        "error": "Authentication failed - check your API key",
                        "address": normalized_address,
                        "timestamp": datetime.now().isoformat(),
                        "status_code": 401,
                        "suggestion": "Verify your API key is correct and active",
                        "api_key_preview": self.api_key[:10] + "..." if len(self.api_key) > 10 else "short_key"
                    }
            
            return {
                "error": f"HTTP error: {status_code}",
                "address": normalized_address,
                "timestamp": datetime.now().isoformat(),
                "details": error_details[:300] if error_details else "No details available"
            }
        except requests.exceptions.RequestException as e:
            return {
                "error": f"Request failed: {str(e)}",
                "address": normalized_address,
                "timestamp": datetime.now().isoformat()
            }
    
    def _process_risk_data(self, raw_data: Dict, address: str) -> Dict:
        """
        Process and enrich the raw API response
        
        Args:
            raw_data: Raw response from Webacy API
            address: The analyzed address
            
        Returns:
            Dict: Processed risk data with additional insights
        """
        processed = {
            "address": address,
            "timestamp": datetime.now().isoformat(),
            "raw_data": raw_data
        }
        
        # Extract key risk metrics
        if isinstance(raw_data, dict):
            # Overall risk assessment
            processed["risk_assessment"] = {
                "overall_score": raw_data.get("overallRisk", 0),
                "risk_level": self._categorize_risk(raw_data.get("overallRisk", 0)),
                "total_issues": raw_data.get("count", 0),
                "medium_risk_issues": raw_data.get("medium", 0),
                "high_risk_issues": raw_data.get("high", 0),
                "is_contract": raw_data.get("isContract", False)
            }
            
            # Extract specific risk factors
            issues = raw_data.get("issues", [])
            if issues:
                processed["risk_factors"] = self._extract_risk_factors(issues)
            
            # Token-specific data if available
            token_data = raw_data.get("tokenData", {})
            if token_data:
                processed["token_analysis"] = self._extract_token_insights(token_data)
        
        return processed
    
    def _categorize_risk(self, score: float) -> str:
        """Categorize risk score into levels"""
        if score < 2:
            return "Low"
        elif score < 5:
            return "Medium"
        elif score < 8:
            return "High"
        else:
            return "Critical"
    
    def _extract_risk_factors(self, issues: list) -> Dict:
        """Extract and categorize risk factors"""
        risk_factors = {
            "critical_risks": [],
            "moderate_risks": [],
            "low_risks": [],
            "positive_indicators": []
        }
        
        for issue in issues:
            if isinstance(issue, dict):
                severity = issue.get("score", 0)
                tags = issue.get("tags", [])
                if tags and isinstance(tags[0], dict):
                    risk_item = {
                        "name": tags[0].get("name", "Unknown"),
                        "description": tags[0].get("description", ""),
                        "severity_score": severity,
                        "risk_type": tags[0].get("type", "unknown"),
                        "key": tags[0].get("key", "unknown")
                    }
                    
                    if severity >= 1.0:
                        risk_factors["critical_risks"].append(risk_item)
                    elif severity >= 0.5:
                        risk_factors["moderate_risks"].append(risk_item)
                    elif severity > 0:
                        risk_factors["low_risks"].append(risk_item)
                    else:
                        risk_factors["positive_indicators"].append(risk_item)
        
        return risk_factors
    
    def _extract_token_insights(self, token_data: Dict) -> Dict:
        """Extract token-specific insights"""
        insights = {}
        
        # Buy/sell taxes
        buy_sell_taxes = token_data.get("buy_sell_taxes", {})
        insights["has_buy_tax"] = buy_sell_taxes.get("has_buy_tax", False)
        insights["has_sell_tax"] = buy_sell_taxes.get("has_sell_tax", False)
        
        # Holder information
        holders = token_data.get("holders", {})
        insights["holder_concentration"] = holders.get("percentageHeldByTop10", 0)
        
        # Developer activity
        insights["recent_dev_activity"] = token_data.get("dev_launched_tokens_in_24_hours", 0)
        
        # Access control
        access_control = token_data.get("access_control", {})
        insights["active_role_holders"] = len(access_control.get("activeRoleHolders", []))
        
        return insights
    
    def batch_analyze(self, addresses: list, delay_between_requests: float = 0.2) -> Dict[str, Dict]:
        """
        Analyze multiple addresses in batch
        
        Args:
            addresses: List of addresses to analyze
            delay_between_requests: Delay in seconds between API calls
            
        Returns:
            Dict: Results keyed by address
        """
        results = {}
        
        print(f"Analyzing {len(addresses)} addresses...")
        for i, address in enumerate(addresses):
            if isinstance(address, str):
                print(f"Processing {i+1}/{len(addresses)}: {address}")
                results[address] = self.get_address_risk_data(address)
                
                # Delay between requests to be API-friendly
                if i < len(addresses) - 1:  # Don't delay after the last request
                    time.sleep(delay_between_requests)
        
        return results
    
    def get_cache_stats(self) -> Dict:
        """Get cache statistics"""
        now = datetime.now()
        valid_entries = sum(1 for addr, data in self.cache.items() 
                          if now - data['timestamp'] < self.cache_duration)
        
        return {
            "total_cached_addresses": len(self.cache),
            "valid_cached_entries": valid_entries,
            "cache_hit_ratio": f"{valid_entries}/{len(self.cache)}" if self.cache else "0/0",
            "cache_duration_minutes": self.cache_duration.total_seconds() / 60
        }
    
    def clear_cache(self):
        """Clear the address cache"""
        self.cache.clear()
    
    def update_api_key(self, new_api_key: str):
        """
        Update API key and refresh session headers
        
        Args:
            new_api_key: New API key to use
        """
        self.api_key = new_api_key
        self.session.headers.update({
            "x-api-key": new_api_key
        })
        # Clear cache when API key changes
        self.clear_cache()
    
    def export_results(self, results: Dict, filename: str = None) -> str:
        """
        Export analysis results to JSON file
        
        Args:
            results: Results dictionary from batch_analyze or single analysis
            filename: Optional filename, auto-generated if not provided
            
        Returns:
            str: Filename of exported file
        """
        if filename is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"webacy_risk_analysis_{timestamp}.json"
        
        with open(filename, 'w') as f:
            json.dump(results, f, indent=2, default=str)
        
        return filename


# Usage example and helper functions
def create_tracker(api_key: str, cache_minutes: int = 30) -> UniswapV3RiskTracker:
    """Factory function to create a tracker instance"""
    return UniswapV3RiskTracker(api_key, cache_minutes)

def analyze_address(tracker: UniswapV3RiskTracker, address: str, show_details: bool = True) -> Dict:
    """Helper function to analyze and display results for an address"""
    result = tracker.get_address_risk_data(address)
    
    if "error" in result:
        print(f"❌ Error analyzing {address}: {result['error']}")
        if "suggestion" in result:
            print(f"💡 Suggestion: {result['suggestion']}")
        return result
    
    if show_details:
        print(f"\n🔍 === Risk Analysis for {address} ===")
        risk_assessment = result.get("risk_assessment", {})
        overall_score = risk_assessment.get('overall_score', 'N/A')
        risk_level = risk_assessment.get('risk_level', 'N/A')
        
        # Risk level emoji
        risk_emoji = {
            'Low': '🟢',
            'Medium': '🟡', 
            'High': '🟠',
            'Critical': '🔴'
        }.get(risk_level, '⚪')
        
        print(f"{risk_emoji} Overall Risk Score: {overall_score}")
        print(f"📊 Risk Level: {risk_level}")
        print(f"🏷️ Address Type: {'Contract' if risk_assessment.get('is_contract') else 'EOA'}")
        print(f"⚠️ Total Issues Found: {risk_assessment.get('total_issues', 0)}")
        
        # Show risk factors if available
        risk_factors = result.get("risk_factors", {})
        
        if risk_factors.get("critical_risks"):
            print(f"\n🚨 Critical Risks ({len(risk_factors['critical_risks'])}):")
            for risk in risk_factors["critical_risks"][:3]:  # Show top 3
                print(f"  • {risk['name']}: {risk['description'][:80]}...")
        
        if risk_factors.get("positive_indicators"):
            print(f"\n✅ Positive Indicators ({len(risk_factors['positive_indicators'])}):")
            for indicator in risk_factors["positive_indicators"][:2]:  # Show top 2
                print(f"  • {indicator['name']}")
        
        # Token analysis if available
        token_analysis = result.get("token_analysis", {})
        if token_analysis:
            print(f"\n💰 Token Analysis:")
            if token_analysis.get("has_buy_tax") or token_analysis.get("has_sell_tax"):
                print(f"  • Buy Tax: {'Yes' if token_analysis.get('has_buy_tax') else 'No'}")
                print(f"  • Sell Tax: {'Yes' if token_analysis.get('has_sell_tax') else 'No'}")
            
            concentration = token_analysis.get("holder_concentration", 0)
            if concentration > 0:
                print(f"  • Top 10 Holders Own: {concentration:.1f}% of supply")
    
    return result

def quick_risk_check(api_key: str, address: str) -> str:
    """Quick risk check that returns a simple risk level"""
    tracker = create_tracker(api_key)
    result = tracker.get_address_risk_data(address)
    
    if "error" in result:
        return f"Error: {result['error']}"
    
    risk_assessment = result.get("risk_assessment", {})
    return risk_assessment.get('risk_level', 'Unknown')

# Example usage and testing
if __name__ == "__main__":
    # Load environment variables
    load_dotenv()
    API_KEY = os.getenv('WEBACY_API_KEY')
    
    if not API_KEY: 
        raise ValueError('Please set the WEBACY_API_KEY environment variable.')
    
    print(f"🔑 Using API Key: {API_KEY[:10]}..." if len(API_KEY) > 10 else "short_key")
    
    # Create tracker instance
    tracker = create_tracker(API_KEY)
    
    # Test API connection first
    print("\n🔗 Testing API connection...")
    connection_test = tracker.test_api_connection()
    print(f"Connection Status: {'✅ Success' if connection_test.get('success') else '❌ Failed'}")
    
    if not connection_test.get('success'):
        print(f"Error Details: {connection_test}")
        print("❌ API connection failed. Check your API key and network connection.")
        exit(1)
    
    print("✅ API connection successful!")
    
    # Test addresses
    test_addresses = [
        "0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2",  # WETH contract
        "0xA0b86a33E6411E3b9F15D3D5dCdb9C8A02b4B2B9",  # Example EOA
    ]
    
    # Analyze single addresses
    print("\n" + "="*60)
    print("🔍 INDIVIDUAL ADDRESS ANALYSIS")
    print("="*60)
    
    for address in test_addresses:
        result = analyze_address(tracker, address)
        time.sleep(1)  # Be nice to the API
    
    # Demonstrate batch analysis
    print("\n" + "="*60)
    print("📊 BATCH ANALYSIS")
    print("="*60)
    
    batch_results = tracker.batch_analyze(test_addresses)
    
    print(f"\n📈 Batch Analysis Summary:")
    for addr, result in batch_results.items():
        if "error" not in result:
            risk_level = result.get("risk_assessment", {}).get("risk_level", "Unknown")
            risk_emoji = {'Low': '🟢', 'Medium': '🟡', 'High': '🟠', 'Critical': '🔴'}.get(risk_level, '⚪')
            print(f"  {risk_emoji} {addr}: {risk_level}")
        else:
            print(f"  ❌ {addr}: Error")
    
    # Cache statistics
    cache_stats = tracker.get_cache_stats()
    print(f"\n💾 Cache Statistics:")
    print(f"  • Total cached addresses: {cache_stats['total_cached_addresses']}")
    print(f"  • Valid cached entries: {cache_stats['valid_cached_entries']}")
    print(f"  • Cache duration: {cache_stats['cache_duration_minutes']} minutes")
    
    # Export results
    filename = tracker.export_results(batch_results)
    print(f"\n💾 Results exported to: {filename}")
    
    print(f"\n🎉 Analysis complete! Processed {len(test_addresses)} addresses.")

🔑 Using API Key: DdhFEmvHL8...

🔗 Testing API connection...
Connection Status: ✅ Success
✅ API connection successful!

🔍 INDIVIDUAL ADDRESS ANALYSIS

🔍 === Risk Analysis for 0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2 ===
🟠 Overall Risk Score: 5.47722557505166
📊 Risk Level: High
🏷️ Address Type: Contract
⚠️ Total Issues Found: 1

🔍 === Risk Analysis for 0xA0b86a33E6411E3b9F15D3D5dCdb9C8A02b4B2B9 ===
🟢 Overall Risk Score: 0
📊 Risk Level: Low
🏷️ Address Type: EOA
⚠️ Total Issues Found: 1

✅ Positive Indicators (1):
  • Insufficient Age

📊 BATCH ANALYSIS
Analyzing 2 addresses...
Processing 1/2: 0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
Processing 2/2: 0xA0b86a33E6411E3b9F15D3D5dCdb9C8A02b4B2B9

📈 Batch Analysis Summary:
  🟠 0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2: High
  🟢 0xA0b86a33E6411E3b9F15D3D5dCdb9C8A02b4B2B9: Low

💾 Cache Statistics:
  • Total cached addresses: 2
  • Valid cached entries: 2
  • Cache duration: 30.0 minutes

💾 Results exported to: webacy_risk_analysis_20250915_13

In [ ]:
## @akinthomasbishop
## https://github.com/AKIN-THOMAS/analytic-sages-fastapi-nextjs/tree/main